In [1]:
import os
import cPickle
import sys
import binascii
import multiprocessing as mp
from itertools import chain
#from scapy.all import *
#sys.path.append('hed-dlg/')
from collections import OrderedDict
import numpy as np

sys.setrecursionlimit(100000)

In [2]:
import ast
import json
import subprocess
import sys

def parse_header(line):
    ret_dict = {}
    h = line.split()
    #ret_dict['direction'] = " ".join(h[3:6])

    if h[2] == 'IP6':
        """
        Conditional formatting based on ethernet type.
        IPv4 format: 0.0.0.0.port
        IPv6 format (one of many): 0:0:0:0:0:0.port
        """
        ret_dict['src_port'] = h[3].split('.')[-1]
        ret_dict['src_ip'] = h[3].split('.')[0]

        ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
        ret_dict['dest_ip'] = h[5].split('.')[0]
    else:
        if len(h[3].split('.')) > 4:
            ret_dict['src_port'] = h[3].split('.')[-1]
            ret_dict['src_ip'] = '.'.join(h[3].split('.')[:-1])
        else:
            ret_dict['src_ip'] = h[3]
            ret_dict['src_port'] = ''

        if len(h[5].split('.')) > 4:
            ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
            ret_dict['dest_ip'] = '.'.join(h[5].split('.')[:-1])
        else:
            ret_dict['dest_ip'] = h[5].split(':')[0]
            ret_dict['dest_port'] = ''

    try:
        if ret_dict['src_port'] == '53' or ret_dict['dst_port'] == '53':
            ret_dict['length'] = int(h[-1][1:-1])
        else:
            try:
                ret_dict['length'] = int(line.split(' length ')[1].split(':')[0])
            except:
                ret_dict['length'] = 0
    except:
        try:
            ret_dict['length'] = int(line.split(' length ')[1].split(':')[0])
        except:
            ret_dict['length'] = 0

    return ret_dict

def parse_data(line, length):
    ret_str = ''
    h, d = line.split(':', 1)
    ret_str = d.strip().replace(' ', '')
    if length != 0:
        ret_str = ret_str[:-(2 * length)]
    return ret_str

def process_packet(output):
    # TODO!! throws away the first packet!
    ret_header = {}
    ret_dict = {}
    ret_data = ''
    for line in output:
        line = line.strip()
        if line:
            if line.startswith('0x'):
                data = parse_data(line, ret_header['length'])
                ret_data = ret_data + data
            else:
                try:
                    ret_header = parse_header(line)
                except:
                    pass
                ret_dict.update(ret_header)
                if ret_data != '':
                    ret_dict['data'] = ret_data
                    ret_data = ''
                yield ret_dict
    
def read_pcap(path):
    hex_sessions = OrderedDict()
    proc = subprocess.Popen('tcpdump -nn -tttt -xx -r '+path,
                            shell=True,
                            stdout=subprocess.PIPE)
    for packet in process_packet(proc.stdout):
        a = str(packet)
        a = ast.literal_eval(a)
        if 'data' in a:
            key = (a['src_ip']+":"+a['src_port'], a['dest_ip']+":"+a['dest_port'])
            keys = [k for k in hex_sessions if (k[0] == key[0] and k[1] == key[1]) or (k[0] == key[1] and k[1] == key[0])]
            if len(keys) > 0:
                #hex_sessions[keys[0]].append(a['direction']+a['data'])
                hex_sessions[keys[0]].append(a['data'])
            else:
                #hex_sessions[key] = [a['direction']+a['data']]
                hex_sessions[key] = [a['data']]
    return hex_sessions

In [ ]:
dataPath = '/data/fs4/datasets/pcaps/gregPcaps/'
testy = read_pcap(dataPath+'a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap')

In [ ]:
with open(dataPath+'a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap') as f:
    counter = 0
    for line in f:
        counter += 1
    print counter

In [10]:
dataPath = '/data/fs4/datasets/pcaps/gregPcaps/'
dirList = os.listdir('/data/fs4/datasets/pcaps/gregPcaps/')
dirList

['NESTthermostat-nf-10days-96bytes.pcap',
 'b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan294.pcap',
 'b-dc-24hrs-96bytes-E-FT-SRV-FW1A-2016-08-09_18-16-vlan210.pcap',
 'a-printers-24hrs-96bytes-E-VA-SRV-FW1A-2016-08-09_14-17-vlan34.pcap',
 'a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap',
 'b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan210.pcap',
 'SonySmartTV-nf-10days-96bytes.pcap',
 'TiVoSeries4-nf-10days-96bytes.pcap']

In [ ]:
complicated = dict()
for capture in dirList:
    dictName = capture
    complicated[dictName] = read_pcap(dataPath+capture)
    
    for ses in complicated[dictName].keys():
        paclens = []
        
        for pac in complicated[dictName][ses]:
            paclens.append(len(pac))
        if np.min(paclens)<80:
            del complicated[dictName][ses]
    
    print dictName + '  is done'

with open('complicated.pickle', 'wb') as handle:
    cPickle.dump(complicated, handle)

NESTthermostat-nf-10days-96bytes.pcap  is done


with open('complicated.pickle', 'rb') as unhandle:
     testDict= cPickle.load(unhandle)

In [4]:
for di in testDict.keys():
    print di, "  ", len(testDict[di].keys())

b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan294.pcap    0
a-printers-24hrs-96bytes-E-VA-SRV-FW1A-2016-08-09_14-17-vlan34.pcap    19629
b-dc-24hrs-96bytes-E-QD-SRV-FW1A-2016-08-09_18-16-vlan210.pcap    2
b-dc-24hrs-96bytes-E-FT-SRV-FW1A-2016-08-09_18-16-vlan210.pcap    994
SonySmartTV-nf-10days-96bytes.pcap    147246
TiVoSeries4-nf-10days-96bytes.pcap    2132
a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap    2
NESTthermostat-nf-10days-96bytes.pcap    4926


In [ ]:
testDict['a-fs-24hrs-96bytes-E-ASH-SRV-FW1A-2016-08-09_18-17-vlan40.pcap']

In [8]:
tester = read_pcap('/data/fs4/datasets/pcaps/gregPcaps/NESTthermostat-nf-10days-96bytes.pcap')

In [9]:
len(tester)

4946